In [6]:
from collections import defaultdict
from functools import reduce
import random
import sys
import os

project_root_path = '/Users/lyc/hybrid_recommendation'
user = {}
tv = {}
file_path = os.path.join(project_root_path, 'collaborative_filtering/data/noclassified/score_nolimit.txt')

with open(file_path, "r", encoding="utf-8") as f:
    for line in f.readlines():
        line = line.strip()
        line = line.split(",")
        if line[0] not in user:
            user[line[0]] = []
            user[line[0]].append(line[1])
        else:
            user[line[0]].append(line[1])
        if line[1] not in tv:
            tv[line[1]] = []
            tv[line[1]].append(line[0])
        else:
            tv[line[1]].append(line[0])

In [7]:
#用户数量,电影数量
print("用户数量:%s,节目数量:%s" % (len(user),len(tv)))

用户数量:113230,节目数量:12848


In [8]:
for i in [1,2,3,4,5,6,7,8,9,10]:
    print("观看大于等于%s的人数:%s" % (i,len(list(filter(lambda x: len(x[1]) >= i , user.items())))))

观看大于等于1的人数:113230
观看大于等于2的人数:90880
观看大于等于3的人数:73355
观看大于等于4的人数:59128
观看大于等于5的人数:47615
观看大于等于6的人数:38174
观看大于等于7的人数:30702
观看大于等于8的人数:24471
观看大于等于9的人数:19616
观看大于等于10的人数:15567


In [9]:
#观看数大于6的观众
user_6 = dict(filter(lambda x: len(x[1]) >= 6 , user.items()))

In [10]:
#核心用户观看的节目的覆盖率
temp = []
for item in user_6.items():
    temp.extend(item[1])
print(len(set(temp)) / len(tv) * 100)

93.06506849315068


In [11]:
len(user_6)

38174

In [12]:
#推荐个数
RECOMMEND_NUM = 5
TEST_SIZE = 3

In [13]:
tv_slist = sorted(tv.items(),key=lambda x: len(x[1]),reverse=True)

In [18]:
#最热门的节目
hot_tv = []
for i,item in enumerate(tv_slist):
    if i < RECOMMEND_NUM:
        print("节目名称:%s,观看用户数:%s"%(item[0],len(item[1])))
        hot_tv.append(item[0])

节目名称:6542,观看用户数:17557
节目名称:3765,观看用户数:15220
节目名称:1453,观看用户数:15182
节目名称:8343,观看用户数:8648
节目名称:10661,观看用户数:8394


In [19]:
hot_precision_all = []
hot_recall_all = []
for item in user.items():
    count = 0
    for u_t in item[1][:3]:
        if u_t in hot_tv:
            count += 1
#     for i in [1,2,3]:
#         index = random.randint(0,len(item[1])-1)
#         if item[1][index] in hot_tv:
#             count += 1
    hot_precision_all.append(count / RECOMMEND_NUM)
    hot_recall_all.append(count / TEST_SIZE)

In [20]:
mean_hot_precision = reduce(lambda x,y:x+y,hot_precision_all) / len(hot_precision_all)
mean_hot_recall = reduce(lambda x,y:x+y,hot_recall_all) / len(hot_recall_all)
hot_f1 = (2 * mean_hot_precision * mean_hot_recall) / (mean_hot_precision + mean_hot_recall)
print("热门推荐top%s:" % RECOMMEND_NUM)
print("准确率:%s" % mean_hot_precision)
print("召回率:%s" % mean_hot_recall)
print("f1-score:%s" % hot_f1)

热门推荐top5:
准确率:0.06963702199060011
召回率:0.11606170331776516
f1-score:0.08704627748827778


In [21]:
tv_all = [item[0] for item in tv.items()]

In [22]:
random_precision_all = []
random_recall_all = []
for item in user.items():
    count = 0
    random_recommend = random.sample(tv_all,RECOMMEND_NUM)
    for u_t in item[1]:
        if u_t in random_recommend:
            count += 1
    random_precision_all.append(count / RECOMMEND_NUM)
    random_recall_all.append(count / TEST_SIZE)

In [23]:
mean_random_precision = reduce(lambda x,y:x+y,random_precision_all) / len(random_precision_all)
mean_random_recall = reduce(lambda x,y:x+y,random_recall_all) / len(random_recall_all)
random_f1 = (2 * mean_random_precision * mean_random_recall) / (mean_random_precision + mean_random_recall)
print("随机推荐%s个节目" % RECOMMEND_NUM)
print("准确率:%s" % mean_random_precision)
print("召回率:%s" % mean_random_recall)
print("f1-score:%s" % random_f1)

随机推荐5个节目
准确率:0.000372692749271395
召回率:0.0006211545821189915
f1-score:0.00046586593658924375
